In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path("..").resolve()))

In [2]:
import torch
from torch.utils.data import DataLoader, ConcatDataset
from models.models import build_model     
from datasets.loader import DataModule, DataConfig  
from train.trainer_v2 import Trainer, TrainConfig  
from train.eval import Evaluator             
from datasets.base import collate_bb

/home/neoph/dev/Train/.tenv312/lib/python3.12/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
/home/neoph/dev/Train/.tenv312/lib/python3.12/site-packages/onnxscript/converter.py:823: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()


In [6]:
import mlflow
mlflow.set_tracking_uri("file:///media/sdb1/mlflow")
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
NUM_CLASSES = 1 + 24
model_uri = "file:///media/sdb1/mlflow/753485487056022103/a7b0ebbbad47442c841112c6bfb35e16/artifacts/model"
model = mlflow.pytorch.load_model(model_uri)

def allow_missing_masks(model):
    orig_forward = model.forward
    def forward(images, targets=None):
        if model.training and targets is not None and not all(("masks" in t) for t in targets):
            rh = model.roi_heads
            saved = (rh.mask_roi_pool, rh.mask_head, rh.mask_predictor)
            try:
                rh.mask_roi_pool, rh.mask_head, rh.mask_predictor = None, None, None
                return orig_forward(images, targets)
            finally:
                rh.mask_roi_pool, rh.mask_head, rh.mask_predictor = saved
        return orig_forward(images, targets)
    model.forward = forward
    return model
model = allow_missing_masks(model)

conf = TrainConfig.from_json("./train_9.json")
trainer = Trainer(model, conf)

In [7]:
def infinite(loader):
    while True:
        for batch in loader:
            yield batch

class AlternatingLoader:
    def __init__(self, loader_a, loader_b, steps=None, start="a"):
        self.a = loader_a
        self.b = loader_b
        self.steps = steps if steps is not None else 2 * max(len(loader_a), len(loader_b))
        self.start = start

    def __len__(self):
        return self.steps

    def __iter__(self):
        ia, ib = infinite(self.a), infinite(self.b)
        for i in range(self.steps):
            if (i % 2 == 0) == (self.start == "a"):
                yield next(ia)
            else:
                yield next(ib)

dm = DataModule(DataConfig(val_frac=0.1, batch_size=conf.batch_size, num_workers=conf.num_workers), with_masks = False)
b_train_loader, b_val_loader = dm.make_loaders_b()

a_train_loader = DataLoader(dm.ds_a_train, batch_size=conf.batch_size, shuffle=True, num_workers=conf.num_workers, collate_fn=collate_bb)
a_val_loader   = DataLoader(dm.ds_a_val,   batch_size=conf.batch_size, shuffle=False, num_workers=conf.num_workers, collate_fn=collate_bb)

mix_train = AlternatingLoader(a_train_loader, b_train_loader)
mix_val   = AlternatingLoader(a_val_loader,   b_val_loader)

In [ ]:
hist_ft = trainer.run(mix_train, mix_val, experiment_name="Att_FT2_Train9")     

2025/12/19 06:01:58 INFO mlflow.tracking.fluent: Experiment with name 'Att_FT2_Train9' does not exist. Creating a new experiment.


[epoch 001/040] step 50/2250 loss 0.9260
[epoch 001/040] step 100/2250 loss 0.5300
[epoch 001/040] step 150/2250 loss 0.6465
[epoch 001/040] step 200/2250 loss 0.4262
[epoch 001/040] step 250/2250 loss 0.3665
[epoch 001/040] step 300/2250 loss 0.3969
[epoch 001/040] step 350/2250 loss 0.7200
[epoch 001/040] step 400/2250 loss 0.6539
[epoch 001/040] step 450/2250 loss 0.4258
[epoch 001/040] step 500/2250 loss 0.4117
[epoch 001/040] step 550/2250 loss 0.5626
[epoch 001/040] step 600/2250 loss 0.3577
[epoch 001/040] step 650/2250 loss 0.4470
[epoch 001/040] step 700/2250 loss 0.3162
[epoch 001/040] step 750/2250 loss 0.5005
[epoch 001/040] step 800/2250 loss 0.4036
[epoch 001/040] step 850/2250 loss 0.4890
[epoch 001/040] step 900/2250 loss 0.6818
[epoch 001/040] step 950/2250 loss 0.4634
[epoch 001/040] step 1000/2250 loss 0.4617
[epoch 001/040] step 1050/2250 loss 0.4653
[epoch 001/040] step 1100/2250 loss 0.4600
[epoch 001/040] step 1150/2250 loss 0.5563
[epoch 001/040] step 1200/2250 

In [6]:
import torch
torch.save(model.state_dict(), "../weights/maskrcnn_attfpn30.pth")